In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,precision_score, recall_score

In [2]:
## 设置字符集，防止中文乱码
mpl.rcParams['font.sans-serif']=[u'simHei']
mpl.rcParams['axes.unicode_minus']=False

In [3]:
# jupyter展示图片，非内嵌显示
# tk: 显示出来，inline：内嵌显示，默认为inline
%matplotlib tk

In [4]:
# 1. 文件数据读取
df = pd.read_csv('../data/result_process02', sep=',')
# 如果有某值为nan，进行删除操作
df.dropna(axis=0, how='any', inplace=True)
df.head(10)

,label,has_date,jieba_cut_content,content_sema
0,1.0,1,非 财务 纠淼 牟 莆 窆 芾 - （ 沙盘 模拟 ） - - ...,8.456151
1,0.0,1,讲 的 是 孔子 后人 的 故事 。 一个 老 领导 回到 家乡 ...,7.486084
2,1.0,1,尊敬 的 贵 公司 ( 财务 / 经理 ) 负责人 您好 ！ 我 ...,7.175171
3,1.0,1,贵 公司 负责人 ( 经理 / 财务 ） 您好 ： 深圳市 华龙 公...,7.565682
4,1.0,1,这是 一封 HTML 格式 信件 ！ - - - - - - - ...,2.063409
5,1.0,1,TO ： 贵 公司 经理 、 财务 您好 ！ 深圳市 春洋 贸易 有...,7.143747
6,0.0,1,那 他 为什么 不 愿意 起诉 ， 既然 这样 了 ！ 起诉 后 ...,4.807568
7,1.0,1,尊敬 的 负责人 （ 经理 ／ 财务 ） ： 您好 ！ 我 是 深...,6.593684
8,1.0,1,您好 以下 是 特别 为 阁下 发 的 香港 信息 ( 图片 ...,7.611074
9,0.0,1,我 觉得 ， 负债 不要紧 ， 最 重要 的 是 能 负得起 这个 ...,7.041340


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64284 entries, 0 to 64619
Data columns (total 4 columns):
label                64284 non-null float64
has_date             64284 non-null int64
jieba_cut_content    64284 non-null object
content_sema         64284 non-null float64
dtypes: float64(2), int64(1), object(1)
memory usage: 2.5+ MB


In [6]:
# 2. 数据分割
x_train, x_test, y_train, y_test = train_test_split(df[['has_date','jieba_cut_content','content_sema']], df['label'], test_size=0.2, random_state=0)
print("训练数据集大小:%d" % x_train.shape[0])
print("测试数据集大小:%d" % x_test.shape[0])

训练数据集大小:51427
测试数据集大小:12857


In [7]:
x_train.head(5)

,has_date,jieba_cut_content,content_sema
18810,1,尊敬 的 公司 （ 工厂 ） 经理 负责人 你好 ： 我 公司 是 ...,6.849239
61632,1,声音 变换器 ： 适用 于 不同 型号 的 手机 ， 电话 ， 网络...,7.242064
5822,1,http : / / tengyingge . blogchina . co...,8.289843
5881,1,刚 开始 ， 是 会 有点 不 习惯 的 。 慢慢来 。 ps ：...,7.063309
26338,1,尊敬 的 公司 （ 工厂 ） 经理 负责人 你好 ： 我 公司 是 ...,6.700159


In [8]:
# 3. 开始模型训练
# 3.1 特征工程，将文本数据转换为数值型数据
vectorizer = CountVectorizer()
transfromer = TfidfTransformer(norm='l2', use_idf=True)
svd = TruncatedSVD(n_components=20)
jieba_cut_content = list(x_train['jieba_cut_content'].astype('str'))
vectorizer_model = vectorizer.fit(jieba_cut_content)
df0 = vectorizer_model.transform(jieba_cut_content)
transfromer_model = transfromer.fit(df0)
df1 = transfromer_model.transform(df0)
svd_model = svd.fit(df1)
df2 = svd_model.transform(df1)
data = pd.DataFrame(df2)
print(data.head(5))
print(data.info())

         0         1         2         3         4         5         6   \
0  0.340289  0.220013 -0.237477 -0.039935 -0.070164  0.002534 -0.160746   
1  0.014968  0.064740  0.026999 -0.004679  0.008155  0.007476  0.016440   
2  0.011862  0.072032  0.047003  0.001431 -0.002081  0.035890  0.009202   
3  0.019714  0.117440  0.132451  0.060417 -0.103507 -0.074860 -0.012283   
4  0.346257  0.215338 -0.234029 -0.043113 -0.071077 -0.000005 -0.162564   

         7         8         9         10        11        12        13  \
0 -0.009913  0.201732 -0.009980 -0.199642  0.038005 -0.065479  0.433020   
1  0.046136  0.029660  0.000079  0.007700  0.023242  0.041449 -0.012425   
2 -0.012193 -0.016869  0.000218  0.001076 -0.003810 -0.002337 -0.003408   
3 -0.005396 -0.027690  0.000261 -0.016997 -0.013367 -0.003378  0.017031   
4 -0.009193  0.199037 -0.009757 -0.195281  0.036327 -0.061918  0.421219   

         14        15        16        17        18        19  
0 -0.476129  0.203667  0.095763 -0

In [9]:
# 3.2 数据合并
data['has_date'] = list(x_train['has_date'])
data['content_sema'] = list(x_train['content_sema'])
print(data.head(5))
print(data.info())

          0         1         2         3         4         5         6  \
0  0.340289  0.220013 -0.237477 -0.039935 -0.070164  0.002534 -0.160746   
1  0.014968  0.064740  0.026999 -0.004679  0.008155  0.007476  0.016440   
2  0.011862  0.072032  0.047003  0.001431 -0.002081  0.035890  0.009202   
3  0.019714  0.117440  0.132451  0.060417 -0.103507 -0.074860 -0.012283   
4  0.346257  0.215338 -0.234029 -0.043113 -0.071077 -0.000005 -0.162564   

          7         8         9      ...             12        13        14  \
0 -0.009913  0.201732 -0.009980      ...      -0.065479  0.433020 -0.476129   
1  0.046136  0.029660  0.000079      ...       0.041449 -0.012425  0.012882   
2 -0.012193 -0.016869  0.000218      ...      -0.002337 -0.003408 -0.003657   
3 -0.005396 -0.027690  0.000261      ...      -0.003378  0.017031  0.002589   
4 -0.009193  0.199037 -0.009757      ...      -0.061918  0.421219 -0.466256   

         15        16        17        18        19  has_date  content_sem

In [10]:
%%time
gb = GradientBoostingClassifier(learning_rate=0.01, n_estimators=100, max_depth=3, min_samples_split=50, loss='deviance', random_state=0)
model = gb.fit(data, y_train)

Wall time: 15.6 s


In [11]:
# 4.1 对测试数据进行特征转换
jieba_cut_content_test = list(x_test['jieba_cut_content'].astype('str'))
data_test = pd.DataFrame(svd_model.transform(transfromer_model.transform(vectorizer_model.transform(jieba_cut_content_test))))
data_test['has_date'] = list(x_test['has_date'])
data_test['content_sema'] = list(x_test['content_sema'])
print(data_test.head(5))
print(data_test.info())

          0         1         2         3         4         5         6  \
0  0.537283  0.009050 -0.087589  0.028997  0.000877 -0.032577 -0.157603   
1  0.210902  0.131679 -0.090069 -0.086947 -0.033332  0.039728  0.007019   
2  0.045151  0.277283  0.233895  0.051083 -0.073983 -0.016347  0.010428   
3  0.032907  0.163067  0.113481  0.002463  0.053731  0.008258  0.004319   
4  0.035278  0.065204  0.026076 -0.023768  0.072914 -0.005830 -0.003035   

          7         8         9      ...             12        13        14  \
0  0.009033 -0.134335  0.009494      ...       0.080454  0.007724 -0.015479   
1 -0.008006  0.011691 -0.000060      ...      -0.025969  0.082347 -0.057375   
2 -0.015062  0.099264 -0.002426      ...      -0.067404 -0.036878 -0.033525   
3 -0.020379  0.117918 -0.002400      ...      -0.062017 -0.058567 -0.012928   
4 -0.008104  0.008742  0.000905      ...      -0.003121 -0.005376 -0.003410   

         15        16        17        18        19  has_date  content_sem

In [12]:
# 4.2 对测试数据进行预测
y_predict = model.predict(data_test)

In [13]:
# 5. 效果评估
print("准确率为:%.5f" % precision_score(y_test, y_predict))
print("召回率为:%.5f" % recall_score(y_test, y_predict))
print("F1值为:%.5f" % f1_score(y_test, y_predict))

准确率为:0.94986
召回率为:0.98866
F1值为:0.96887
